<br>
<h1><b>Query Optimization Checker </h1>
<h3> Answer the question of what queries are driving clicks / impressions but are not found in <b>{{ PAGE SECTION }}</b>? </h3> 
<h3> Use 'xpath_selector' variable to define the <b>{{ PAGE SECTION }}</b>
</h3> 
<br>


In [1]:
#@title Mount Drive to the Notebook { display-mode: "form" }
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.insert(0, "drive/My Drive/Colab Notebooks/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<h1> Crawl & GSC Data</h1>

In [2]:
#@title Import dependencies { display-mode: "form" }
import pandas
from api import *
from datetime import datetime
from bs4 import BeautifulSoup, NavigableString
import requests
import random
import lxml
import lxml.html
import lxml.etree
from lxml.etree import ParseError
from lxml.etree import ParserError
from urllib.parse import urlparse
import os
from google.colab import data_table

**<h2> Input Variables - Only Cells to Edit!</h2>**
<p>After Adding Input Variables Use 'CTRL + fn9' to start running script (or go to "Runtime" > "Run All")</p>

In [3]:
#@title Fill out all input variables... { run: "auto", display-mode: "form" }
colab_path = "/content/drive/My Drive/Colab Notebooks/" #@param {type: "string"}
#@markdown ---
domain_lookup = 'https://www.inseev.com/'  #@param {type: "string"}
#@markdown ---
#@markdown ### For "domain properties" use format "sc-domain:domain.com"
gsc_property = 'https://www.inseev.com/'  #@param {type: "string"}
startdate = '2021-02-01'  #@param {type: "date"}
enddate = '2021-03-13'  #@param {type: "date"}
gsc_sorting_field = "clicks_sum"   #@param ['clicks_sum', 'impressions_sum']
gsc_limit_pages_number =   40#@param {type: "number"}
#@markdown ---
#@markdown ### Set an XPath selector if you want to scrape part of the HTML page content
xpath_selector = "//title" #@param {type: "string"}
#@markdown ---
#@markdown ### Set brand query EXCLUSIONS with format *query1|query2|...*  
#@markdown (*Must contain something; if no brand input "xxx")
brand_exclusions = "inseev|insev" #@param {type: "string"}
#@markdown ---
#@markdown ### Set impression LESS THAN X EXCLUSION to remove irrelevant queries
#@markdown (*Must contain something; if no exclusion add "0")
impression_exclusion = "5" #@param {type: "string"}
#@markdown ---
#@markdown ### Set page INCLUSIONS with format *urlID1|urlID2|...*
#@markdown (*Leave blank if no exclusions needed)
page_inclusions = "" #@param {type: "string"}
#@markdown ---


domain_name = domain_lookup.split("www.")[-1].split("//")[-1].split(".")[0]
date = datetime.today().strftime('%Y-%m-%d')

domain_clean = urlparse("{}".format(domain_lookup)).netloc
path_step1 = "{}/{}/".format(colab_path,domain_clean)
path = "{}/{}/Query Optimizer_{}".format(colab_path,domain_clean,date)
path_rawData = "{}/{}/Raw Data (Archive)".format(colab_path,domain_clean,date)
date = datetime.today().strftime('%Y-%m-%d')

try:
    os.mkdir(path_step1)
except OSError:
    print ("Creation of the directory %s failed" % path_step1)
else:
    print ("Successfully created the directory %s " % path_step1)

try:
    os.mkdir(path_rawData)
except OSError:
    print ("Creation of the directory %s failed" % path_rawData)
else:
    print ("Successfully created the directory %s " % path_rawData)

try:
    os.mkdir(path)
except OSError:
    print ("Creation of the directory %s failed" % path)
else:    print ("Successfully created the directory %s " % path)

Successfully created the directory /content/drive/My Drive/Colab Notebooks//www.inseev.com/ 
Successfully created the directory /content/drive/My Drive/Colab Notebooks//www.inseev.com/Raw Data (Archive) 
Successfully created the directory /content/drive/My Drive/Colab Notebooks//www.inseev.com/Query Optimizer_2021-03-21 


<h2>PART 1: Get GSC Data </h2>

In [4]:
#@title Collect data from Google Search Console { display-mode: "form" }
domain_name = domain_lookup.split("www.")[-1].split("//")[-1].split(".")[0]
date = datetime.today().strftime('%Y-%m-%d')

# Options: 'date,' 'device,' 'page,' , 'query' and "country"
dimensions=['page','query']

gsc_df = gscservice.get_site_data(
    gsc_property,
    startdate=startdate,
    enddate=enddate,
    dimensions=dimensions,
    output_fn="{}/{}/Raw Data (Archive)/rawData_{}_{}_{}_by_{}.csv".format(colab_path,domain_clean,domain_name, startdate.replace("-",""), enddate.replace("-",""), '_'.join(dimensions))
)

# Filter to only non-brand && specified page type (if any)
kw_filter = ~gsc_df["query"].str.contains("{}".format(brand_exclusions), case = False, regex=True)
gsc_df = gsc_df[kw_filter]
filter_pageType = gsc_df["page"].str.contains("{}".format(page_inclusions), case = False, regex=True)
gsc_df = gsc_df[filter_pageType]

# filter_pageTypeExclude = ~gsc_df["page"].str.contains("{}".format(pageType_exclude_ngram), case = False, regex=True)
# gsc_df = gsc_df[filter_pageTypeExclude]


# Insert domain & datasetID
gsc_df.insert(loc = 2, column = "domain", value = domain_lookup)
datasetID = "{}_{}_{}_by_{}".format(domain_name, startdate.replace("-",""), enddate.replace("-",""), '_'.join(dimensions))
gsc_df.insert(loc = 0, column = "gsc_datasetID", value = datasetID)


Building new /content/drive/My Drive/Colab Notebooks//www.inseev.com/Raw Data (Archive)/rawData_inseev_20210201_20210313_by_page_query.csv file
Getting Webmaster Service
Pulling Google Search Console Data: 100%|██████████| 41/41 [00:59<00:00,  1.44s/it]
Total rows found: 96064. Saving to csv.


In [5]:
# Preview the export from GSC API
gsc_df.to_csv(os.path.join(path,"step1_raw-page-query-data_{}-{}.csv".format(domain_name,date)),  index=False)
data_table.DataTable(gsc_df, include_index=False, num_rows_per_page=10)

Output hidden; open in https://colab.research.google.com to view.

<h2>PART 2: Aggregate Data </h2>

In [6]:
#@title calculate aggregated metrics and rename dataframe columns { display-mode: "form" }
select_cols = ["gsc_datasetID", "domain", "page", "query", "clicks", "impressions", "ctr", "position"]
gsc_df = gsc_df[select_cols]

# if a query terms list has been defined at the beginning
# the GSC dataset will be filtered by that list

# if len(query_terms_list) > 0:
#     gsc_df = gsc_df.loc[gsc_df['query'] in query_terms_list]

grouped_gsc = (
    gsc_df.groupby(["gsc_datasetID","domain","page"])
    .agg({"clicks": "sum",
          "impressions": "sum",
          "ctr" : "mean",
          "position":["size","max","min","mean"]})
    .reset_index()
    .pipe(lambda x: x.set_axis([f'{a}' if b == '' else f'{a}_{b}' for a,b in x.columns], axis=1, inplace=False))
)

In [7]:
# Preview dataset aggregated by page level
data_table.DataTable(grouped_gsc.head(100), include_index=False, num_rows_per_page=10)

,gsc_datasetID,domain,page,clicks_sum,impressions_sum,ctr_mean,position_size,position_max,position_min,position_mean
0,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,10,60364,0.000228,13196,324,1,65.501970
1,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/#:~:text=Case%20Studies...,0,5,0.000000,5,27,14,18.800000
2,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/#:~:text=We%20Work%20Wi...,0,5,0.000000,5,27,14,18.800000
3,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/5-local-seo-strategies-...,0,6652,0.000000,2175,200,4,79.888736
4,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/avoiding-the-spam-folder/,0,14,0.000000,13,85,28,55.846154
...,...,...,...,...,...,...,...,...,...,...
95,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/webinars/seo-toolbox-3-...,0,155,0.000000,61,200,30,61.590164
96,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/website-seo-audit-2/,0,11,0.000000,10,195,43,84.800000
97,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/website-seo-audit/,41,124870,0.000259,13981,198,1,58.652028
98,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/website-seo-audit/#:~:t...,0,10,0.000000,10,22,6,12.600000


In [8]:
#@title { display-mode: "form" }
gsc_all_data = (
    gsc_df.groupby(["gsc_datasetID","domain","page","query"])
    .agg({"clicks": "sum",
          "impressions": "sum",
          "position":["size","max","min","mean"]})
    .reset_index()
    .pipe(lambda x: x.set_axis([f'{a}' if b == '' else f'{a}_{b}' for a,b in x.columns], axis=1, inplace=False))
    .round(1)
)

filter_impressionsLessThan = gsc_all_data["impressions_sum"] > int("{}".format(impression_exclusion))
gsc_all_data = gsc_all_data[filter_impressionsLessThan]

In [9]:
# Preview dataset aggregated by page and query level
data_table.DataTable(gsc_all_data.head(100), include_index=False, num_rows_per_page=10)

,gsc_datasetID,domain,page,query,clicks_sum,impressions_sum,position_size,position_max,position_min,position_mean
1,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,ad agencies san diego ca,0,9,9,102,82,90.1
2,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,ad agency jobs san diego,0,28,26,100,79,89.1
3,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,ad agency san diego,0,56,39,99,84,90.9
5,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,advertising agencies in san diego,0,9,6,96,86,91.5
6,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,advertising agencies in san diego ca,0,7,7,100,81,92.0
...,...,...,...,...,...,...,...,...,...,...
145,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,digital marketing services san diego,0,136,41,69,47,56.1
146,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,digital media companies in san diego,0,11,11,58,46,52.9
148,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,digital performance marketing agency,0,11,7,110,97,100.7
149,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,digital performance marketing services,0,7,3,101,91,96.0


<h2> Part 3: Scrape Top Pages & Find Query Occurrences </h2>

> Indented block



In [10]:
#@title { display-mode: "form" }
### [BELOW AGGREGATES PAGES FOR SCRAPER]
# Take the first N pages based on the variable 'gsc_sorting_field'
top_n_pages = grouped_gsc.sort_values(by=gsc_sorting_field, ascending=False)
top_n_pages = top_n_pages.reset_index()
top_n_pages = top_n_pages.head(gsc_limit_pages_number)[["gsc_datasetID","domain","page"]]
# Filter the main dataframe by keeping only data from the top N pages defined above
joined_df = gsc_all_data.merge(top_n_pages, on=["gsc_datasetID","domain","page"])

def GET_UA():
  uastrings = [
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/28.0.1500.72 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10) AppleWebKit/600.1.25 (KHTML, like Gecko) Version/8.0 Safari/600.1.25",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",
        "Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.111 Safari/537.36",
        "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_5) AppleWebKit/600.1.17 (KHTML, like Gecko) Version/7.1 Safari/537.85.10",
        "Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko",
        "Mozilla/5.0 (Windows NT 6.3; WOW64; rv:33.0) Gecko/20100101 Firefox/33.0",
        "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/38.0.2125.104 Safari/537.36"
  ]
  return random.choice(uastrings)
crawl_res = []

# for each page in the top N list we are going to scrape the html content
for index, row in top_n_pages.iterrows():
  try:
    # 'filtered_df' contains 'joined_df' data sliced 
    # by the page analyzed during this iteration 
    filtered_df = joined_df.loc[joined_df['page'] == row.to_dict()['page']]
    USER_AGENT = GET_UA()
    headers = {'user-agent': USER_AGENT}
    resp = requests.get(row.to_dict()['page'], headers=headers)
    # parse the HTML with Beautiful soup
    if resp.status_code == 200:
        if xpath_selector is not None and xpath_selector != '':
            tree = lxml.html.fromstring(resp.content)
            # Get element using XPath 
            xpath_selection_content = tree.xpath(xpath_selector)
            selected_content = b'\n'.join([lxml.etree.tostring(elem) for elem in xpath_selection_content])
            bs = BeautifulSoup(selected_content, "html.parser")
        else:
            bs = BeautifulSoup(resp.content, "html.parser")

        # for each query term in the filtered dataframe we find it 
        # in the parsed html code and we count the occurrences
        for i, r in filtered_df.iterrows():
            search_query = r.to_dict()['query']
            search_domain = r.to_dict()['domain']
            search_page = r.to_dict()['page']
            search_gsc_dataset_id = r.to_dict()['gsc_datasetID']
            # find all occurrences for the query term 
            # in the Beautifulsoup parsed page content
            occ = bs.find_all(text=lambda x: x and search_query in x.lower())
            occ_text = []
            # if we have found at least one occurrence of the query term 
            # we just check the Beautifulsoup item class to manage correctly
            # the text where each occurrence has been found
            if len(occ) > 0:
                for o in occ:
                    if isinstance(o, NavigableString):
                        occ_text.append(str(o))
                    else:
                        occ_text.append(o.text)

                crawl_res.append(dict(
                    query=search_query,  # query term we have found in the page content
                    domain=search_domain,  # domain from gsc
                    page_crawl=search_page,  # page from gsc we have scraped
                    gsc_datasetID=search_gsc_dataset_id,  # dataset id from gsc
                    text = occ_text,  # list of texts where we have found the query term
                    occurrences = len(occ_text)  # number of occurrences for the query term
                ))

  # below exceptions stop errors from breaking tool
  except ParserError as pe:
    print("ParserError: Error Message - {0}".format(pe))
    pass

  except BaseException as ge:
    print("Unidentified Error - {0}".format(ge))
    pass


# transform the list of dictionaries into a dataframe to be able to work with the exisiting dataframes
crawl_df = pandas.DataFrame([c for c in crawl_res])

In [11]:
# Preview crawled data
crawl_df.to_csv(os.path.join(path,"step2_query-matches_{}-{}.csv".format(domain_name,date)),  index=False)
data_table.DataTable(crawl_df, include_index=False, num_rows_per_page=10)

,query,domain,page_crawl,gsc_datasetID,text,occurrences
0,audit service,https://www.inseev.com/,https://www.inseev.com/website-seo-audit/,inseev_20210201_20210313_by_page_query,[SEO Audit Services: Complete SEO Website Anal...,1
1,audit services,https://www.inseev.com/,https://www.inseev.com/website-seo-audit/,inseev_20210201_20210313_by_page_query,[SEO Audit Services: Complete SEO Website Anal...,1
2,complete seo,https://www.inseev.com/,https://www.inseev.com/website-seo-audit/,inseev_20210201_20210313_by_page_query,[SEO Audit Services: Complete SEO Website Anal...,1
3,complete seo website analysis,https://www.inseev.com/,https://www.inseev.com/website-seo-audit/,inseev_20210201_20210313_by_page_query,[SEO Audit Services: Complete SEO Website Anal...,1
4,seo audit,https://www.inseev.com/,https://www.inseev.com/website-seo-audit/,inseev_20210201_20210313_by_page_query,[SEO Audit Services: Complete SEO Website Anal...,1
...,...,...,...,...,...,...
68,chrome extensions,https://www.inseev.com/,https://www.inseev.com/top-5-chrome-extensions...,inseev_20210201_20210313_by_page_query,[Top 5 Chrome Extensions for Link Building | I...,1
69,seo site migration,https://www.inseev.com/,https://www.inseev.com/the-correct-way-to-cons...,inseev_20210201_20210313_by_page_query,[The Correct Way to Consult on a SEO Site Migr...,1
70,touch of modern,https://www.inseev.com/,https://www.inseev.com/touch-of-modern/,inseev_20210201_20210313_by_page_query,[Case study page touch of modern - Inseev Inte...,1
71,enterprise link building,https://www.inseev.com/,https://www.inseev.com/webinar/enterprise-link...,inseev_20210201_20210313_by_page_query,[Enterprise Link Building Webinar - Inseev Int...,1


<h2>Part 4: Merge & Prepare Data </h2>

In [12]:
#@title { display-mode: "form" }

output = joined_df.merge(crawl_df, how="left", left_on=["gsc_datasetID", "domain","page","query"], right_on=["gsc_datasetID", "domain","page_crawl","query"])

select_cols_final = ["gsc_datasetID","domain","page","query","clicks_sum","impressions_sum", 
        "position_size","position_max","position_min","position_mean","text","occurrences"]

output = output[select_cols_final]
output = output.rename(
    mapper={
        "gsc_datasetID": "gsc_datasetID",
        "domain": "domain",
        "page": "page_gsc",
        "query": "query_gsc",
        "clicks_sum": "clicks_sum_gsc",
        "impressions_sum": "impressions_sum_gsc",
        "ctr_mean": "ctr_mean_gsc",
        "position_size": "count_instances_gsc",
        "position_max": "position_max_gsc",
        "position_min": "position_min_gsc",
        "position_mean": "position_mean_gsc",
        "text": "text_crawl",
        "occurrences": "occurrences_crawl"
    }, axis="columns")

output.text_crawl = output.text_crawl.fillna(value = "N/A")
output.occurrences_crawl = output.occurrences_crawl.fillna(value = "No Matching Text - Potential Optimization Opp")


In [13]:
output.to_csv(os.path.join(path,"step3_query-optimizer_{}-{}.csv".format(domain_name,date)),  index=False)
data_table.DataTable(output, include_index=False, num_rows_per_page=10)

,gsc_datasetID,domain,page_gsc,query_gsc,clicks_sum_gsc,impressions_sum_gsc,count_instances_gsc,position_max_gsc,position_min_gsc,position_mean_gsc,text_crawl,occurrences_crawl
0,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,ad agencies san diego ca,0,9,9,102,82,90.1,N/A,No Matching Text - Potential Optimization Opp
1,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,ad agency jobs san diego,0,28,26,100,79,89.1,N/A,No Matching Text - Potential Optimization Opp
2,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,ad agency san diego,0,56,39,99,84,90.9,N/A,No Matching Text - Potential Optimization Opp
3,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,advertising agencies in san diego,0,9,6,96,86,91.5,N/A,No Matching Text - Potential Optimization Opp
4,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/,advertising agencies in san diego ca,0,7,7,100,81,92.0,N/A,No Matching Text - Potential Optimization Opp
...,...,...,...,...,...,...,...,...,...,...,...,...
3116,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/wp-content/themes/insee...,audit seo pdf,0,8,7,57,31,51.9,N/A,No Matching Text - Potential Optimization Opp
3117,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/wp-content/themes/insee...,seo audit pdf,0,7,5,55,45,49.0,N/A,No Matching Text - Potential Optimization Opp
3118,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/wp-content/themes/insee...,seo audit report sample pdf,0,14,12,91,45,61.6,N/A,No Matching Text - Potential Optimization Opp
3119,inseev_20210201_20210313_by_page_query,https://www.inseev.com/,https://www.inseev.com/wp-content/themes/insee...,seo audit template pdf,0,66,26,96,65,80.4,N/A,No Matching Text - Potential Optimization Opp
